<a href="https://colab.research.google.com/github/hc2twv/UPSE_OP/blob/main/OCHAvanzado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Optimización de rutas de vehículos de comunicación, como drones o vehículos terrestres

In [4]:
import numpy as np

class AntColonyOptimization:
    def __init__(self, distance_matrix, n_ants=10, n_iterations=100, alpha=1, beta=2, evaporation_rate=0.5):
        """
        Inicializa el algoritmo de Colonias de Hormigas para la optimización de rutas de vehículos de comunicación.

        :param distance_matrix: Matriz de distancias entre los puntos de entrega.
        :param n_ants: Número de hormigas en la colonia.
        :param n_iterations: Número de iteraciones del algoritmo.
        :param alpha: Peso de la feromona en la elección de la siguiente ciudad.
        :param beta: Peso de la distancia en la elección de la siguiente ciudad.
        :param evaporation_rate: Tasa de evaporación de feromonas después de cada iteración.
        """
        self.distance_matrix = distance_matrix
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.pheromone_matrix = np.ones_like(distance_matrix) / np.mean(distance_matrix)  # Inicialización de feromonas

    def run(self):
        """
        Ejecuta el algoritmo ACO para encontrar la mejor ruta de entrega.

        :return: La mejor ruta encontrada y su distancia.
        """
        best_route = None
        best_distance = np.inf
        for _ in range(self.n_iterations):
            routes = self.generate_ant_routes()
            self.update_pheromones(routes)
            shortest_route, shortest_distance = min(routes, key=lambda x: x[1])
            if shortest_distance < best_distance:
                best_route = shortest_route
                best_distance = shortest_distance
        return best_route, best_distance

    def generate_ant_routes(self):
        """
        Genera las rutas de todas las hormigas.

        :return: Lista de todas las rutas generadas por las hormigas.
        """
        routes = []
        for _ in range(self.n_ants):
            route = self.generate_ant_route()
            routes.append(route)
        return routes

    def generate_ant_route(self):
        """
        Genera la ruta de una hormiga.

        :return: Ruta generada por la hormiga y su distancia total.
        """
        current_point = np.random.randint(len(self.distance_matrix))  # Seleccionamos un punto de inicio aleatorio
        unvisited_points = set(range(len(self.distance_matrix)))
        unvisited_points.remove(current_point)
        route = [current_point]
        total_distance = 0
        while unvisited_points:
            next_point = self.choose_next_point(current_point, unvisited_points)
            total_distance += self.distance_matrix[current_point][next_point]
            route.append(next_point)
            unvisited_points.remove(next_point)
            current_point = next_point
        total_distance += self.distance_matrix[route[-1]][route[0]]  # Añadimos la distancia de vuelta al punto inicial
        return route, total_distance

    def choose_next_point(self, current_point, unvisited_points):
        """
        Elige el próximo punto a visitar por una hormiga.

        :param current_point: Punto actual.
        :param unvisited_points: Conjunto de puntos no visitados.
        :return: Próximo punto a visitar.
        """
        probabilities = self.calculate_probabilities(current_point, unvisited_points)
        next_point = np.random.choice(list(unvisited_points), p=probabilities)
        return next_point

    def calculate_probabilities(self, current_point, unvisited_points):
        """
        Calcula las probabilidades de visitar los puntos no visitados desde el punto actual.

        :param current_point: Punto actual.
        :param unvisited_points: Conjunto de puntos no visitados.
        :return: Lista de probabilidades de visitar cada punto.
        """
        pheromones = self.pheromone_matrix[current_point, list(unvisited_points)]
        distances = self.distance_matrix[current_point, list(unvisited_points)]
        visibility = 1 / distances
        total_pheromone = np.sum(pheromones ** self.alpha * visibility ** self.beta)
        probabilities = (pheromones ** self.alpha * visibility ** self.beta) / total_pheromone
        return probabilities

    def update_pheromones(self, routes):
        """
        Actualiza las feromonas en la matriz de feromonas basándose en las rutas de las hormigas.

        :param routes: Lista de rutas de las hormigas.
        """
        self.pheromone_matrix *= (1 - self.evaporation_rate)  # Evaporación de feromonas
        for route, distance in routes:
            for i in range(len(route) - 1):
                self.pheromone_matrix[route[i], route[i+1]] += 1 / distance
                self.pheromone_matrix[route[i+1], route[i]] += 1 / distance  # Asegurar simetría


# Ejemplo de uso:
# Creamos una matriz de distancias de ejemplo (por ejemplo, distancias entre puntos de entrega)
distance_matrix = np.array([[0, 10, 20, 15],
                            [10, 0, 25, 30],
                            [20, 25, 0, 35],
                            [15, 30, 35, 0]])

# Creamos una instancia del algoritmo ACO para optimizar las rutas de vehículos de comunicación
aco = AntColonyOptimization(distance_matrix)

# Ejecutamos el algoritmo para encontrar la mejor ruta
best_route, best_distance = aco.run()
print(f"La mejor ruta encontrada es: {best_route}, con una distancia total de {best_distance}.")



La mejor ruta encontrada es: [2, 1, 0, 3], con una distancia total de 85.
